In [ ]:
"""
look into resnet
"""

In [2]:
import os
import sys
# add parent directory to path
sys.path.append(os.path.abspath("../.."))
print(sys.path)

import torch
from models.resnet import ResNetCond

['/BS/data_mani_compress/work/miniforge3/envs/dgm_geometry/lib/python311.zip', '/BS/data_mani_compress/work/miniforge3/envs/dgm_geometry/lib/python3.11', '/BS/data_mani_compress/work/miniforge3/envs/dgm_geometry/lib/python3.11/lib-dynload', '', '/BS/data_mani_compress/work/miniforge3/envs/dgm_geometry/lib/python3.11/site-packages', '/BS/data_mani_compress/work/thesis/thesis/external/guided-diffusion', '/BS/data_mani_compress/work/thesis/thesis/external/DenseFlow', '/BS/data_mani_compress/work/thesis/thesis']


In [3]:
resnet_classifier = ResNetCond(num_classes=256)
# print number of parameters
num_params = sum(p.numel() for p in resnet_classifier.parameters())
print(f"Number of parameters in ResNetCond classifier: {num_params}")

resnet_regressor = ResNetCond(task_type='regression')
# print number of parameters
num_params = sum(p.numel() for p in resnet_regressor.parameters())
print(f"Number of parameters in ResNetCond regressor: {num_params}")

# find size in GB of the model
model_size = sum(p.numel() * p.element_size() for p in resnet_classifier.parameters()) / (1024 ** 3)
print(f"Size of ResNetCond classifier model: {model_size:.4f} GB")

model_size = sum(p.numel() * p.element_size() for p in resnet_regressor.parameters()) / (1024 ** 3)
print(f"Size of ResNetCond regressor model: {model_size:.4f} GB")

Number of parameters in ResNetCond classifier: 24041888
Number of parameters in ResNetCond regressor: 23511233
Size of ResNetCond classifier model: 0.0896 GB
Size of ResNetCond regressor model: 0.0876 GB


In [ ]:
import time
import torch

resnet_regressor.eval().to('cuda')

# test with different batch sizes
batch_sizes_to_test = [4, 8, 16]
for batch_size in batch_sizes_to_test:
    input_tensor = torch.randn(batch_size, 3, 224, 224).to('cuda')
    condition_tensor = torch.randn(batch_size).to('cuda')

    # warm-up
    with torch.no_grad():
        for _ in range(10):
            _ = resnet_regressor(input_tensor, condition_tensor)

    # timing
    torch.cuda.synchronize()  # IMPORTANT if using GPU
    start = time.time()

    with torch.no_grad():
        for _ in range(100):
            _ = resnet_regressor(input_tensor, condition_tensor)

    torch.cuda.synchronize()
    end = time.time()

    avg_time = (end - start) / 100
    print(f"Average forward pass time per sample: {avg_time/batch_size:.6f} seconds for batch size {batch_size}")


Average forward pass time per sample: 0.001051 seconds for batch size 4
Average forward pass time per sample: 0.000766 seconds for batch size 8
Average forward pass time per sample: 0.000710 seconds for batch size 16


: 

In [ ]:
# for regressor:
# Average forward pass time per sample: 0.000594 seconds for batch size 1024
# Average forward pass time per sample: 0.000600 seconds for batch size 1340
# Average forward pass time per sample: 0.000597 seconds for batch size 1564

# Average forward pass time per sample: 0.001051 seconds for batch size 4
# Average forward pass time per sample: 0.000766 seconds for batch size 8
# Average forward pass time per sample: 0.000710 seconds for batch size 16

# for classifier:
# Average forward pass time per sample: 0.000594 seconds for batch size 1024
# Average forward pass time per sample: 0.000600 seconds for batch size 1340
# Average forward pass time per sample: 0.000597 seconds for batch size 1564

# Average forward pass time per sample: 0.001143 seconds for batch size 4
# Average forward pass time per sample: 0.000769 seconds for batch size 8
# Average forward pass time per sample: 0.000712 seconds for batch size 16